In [6]:
from shapely.geometry import shape
import rasterio
from src.imagery import Collection
from rasterio.plot import show
from rasterio.mask import mask
from rasterio.crs import CRS
from rasterio.merge import merge
from rasterio.fill import fillnodata
from rasterio.transform import Affine
from rasterio.warp import reproject, Resampling, transform_geom
from rasterio import features
from rasterio.features import geometry_mask, rasterize
import glob
import os
import numpy as np
import matplotlib.pyplot as plt

In [72]:
def get_cloud_mask(band=None, res=None):
    if band is not None and res is None:
        pattern = f'*_{band}.jp2'
    
    if res is not None and band is None:
        pattern = f'*_{res}m.jp2'
        
    # obtiene la banda de cada tile
    search_criteria = f"*.SAFE/GRANULE/*/QI_DATA/{pattern}"
    q = os.path.join(dirpath, search_criteria)
    dem_fps = glob.glob(q)

    # load source datasets
    files = []
    for fp in dem_fps:
        src = rasterio.open(fp)
        files.append(src)

    return files


In [73]:
maskpath = "/home/mluzu/Downloads/S2A_MSIL2A_20210916T141051_N0301_R110_T20HMH_20210916T175608.SAFE/GRANULE/L2A_T20HMH_A032568_20210916T142000/QI_DATA/MSK_CLDPRB_20m.jp2"
imgpath = "/home/mluzu/Downloads/S2A_MSIL2A_20210916T141051_N0301_R110_T20HMH_20210916T175608.SAFE/GRANULE/L2A_T20HMH_A032568_20210916T142000/IMG_DATA/R20m/T20HMH_20210916T141051_B02_20m.jp2"

In [74]:
cloud_masks = rasterio.open(maskpath)
img = rasterio.open(imgpath)
dst = rasterio.open("/home/mluzu/Downloads/", 'w', **img.meta)

In [79]:
#mask = cloud_masks.read()
#mask = np.where(mask > 50, True,False)
# masked_imag = np.ma.masked_array(image, mask=mask)
mask_arr = np.ma.masked_greater_equal(cloud_masks.read(1), 50)

(5490, 5490)

In [76]:
img_arr = img.read(1)

In [86]:
dst.write_mask(1, mask_arr)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()